<a href="https://colab.research.google.com/github/JustinJiangNext/AI-Image-Detection-Benchmarking/blob/main/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
#install libraries
!pip install torch torchvision

In [89]:
#import libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision import models


In [90]:
#@title Define Hyperparameters
num_classes = 10
num_epochs = 10
batch_size = 100
lr = 1e-3 #learning rate

In [91]:
#@title Downloading the data and loading into Pytorch DataLoader object

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

if torch.cuda.is_available():
  train_data.train_data.cuda()
  train_data.train_labels.cuda()
  test_data.train_data.cuda()
  test_data.train_labels.cuda()

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [96]:
#@title Define model class

class LinearNN(nn.Module):
  def __init__(self):
    super(LinearNN, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 128)  # First fully connected layer
    self.fc2 = nn.Linear(128, 10)  # Second fully connected layer

  def forward(self, x):
    x = torch.flatten(x, start_dim=1)  # Flatten the image
    x = F.relu(self.fc1(x))  # Apply ReLU activation
    x = self.fc2(x)  # Apply the second linear layer
    x = F.softmax(x, dim=1)  # Apply softmax activation for class probabilities
    return x  # Output probabilities

class ConvolutionalNN(nn.Module):
  def __init__(self):
        super(ConvolutionalNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)  # First convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # Second convolutional layer
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Second fully connected layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer

  def forward(self, x):
      x = F.relu(self.conv1(x))  # Apply ReLU after first conv layer
      x = self.pool(x)  # Apply max pooling
      x = F.relu(self.conv2(x))  # Apply ReLU after second conv layer
      x = self.pool(x)  # Apply max pooling
      x = torch.flatten(x, start_dim=1)  # Flatten for the fully connected layers
      x = F.relu(self.fc1(x))  # Apply ReLU activation to first fully connected layer
      x = self.fc2(x)  # Output layer (no softmax needed if using CrossEntropyLoss)
      return x  # Output logits


class DenseNetNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the pre-trained DenseNet121 model
        self.densenet = models.densenet121(pretrained=True)

        # Modify the first convolution layer to accept 1-channel input (grayscale)
        self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size = 3, padding = 1)

        # Replace the classifier with a new one for 10 classes (MNIST)
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, 10)

    def forward(self, x):
        print(x.size())
        return self.densenet(x)




In [97]:
#@title Build the model

net = DenseNetNN()
if torch.cuda.is_available():
  net.cuda()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [98]:
#@title Training the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

# Training loop
net.train()
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train_gen):
        images, labels = images.cuda(), labels.cuda()

        # Forward pass
        print(images.size())
        outputs = net(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (i+1) % 10 == 0 or (i+1) == len(train_gen):
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_gen)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}] Average Loss: {total_loss/len(train_gen):.4f}")

Streaming output truncated to the last 5000 lines.
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
Epoch [7/10], Step [30/600], Loss: 0.0086
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Si

In [99]:
# Evaluation loop
net.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_gen:
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test data: {100 * correct / total:.2f}%")


torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([10